In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-11 18:57:15.657157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744397835.673603 2429702 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744397835.677761 2429702 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744397835.689430 2429702 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, peft, pickle, pytest, ipytest, pd, transformers, torch, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_builder1, ds_builder2, ds_builder3, ds_gs, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_with_strings, map_data, print_trainable_parameters, metric, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

## Dataset value example 
{'record': 'NZ_LIXJ01000001.1', 'text': 'CGACTCCGCACAGGCCCTAGGCGTGGTCCACGCATCGAGCGCGTGCGCCGTGGCATACACCCGGGCCACCCGTACGACCCTTCCGGCCGCCCCGTGTGCGCCCTCGTCGCCGAGTCCGCTCCGGCGCGCGCGGGCGCACTGCTGTTCCGCGCGGTGTCATGCGCTCCTCCCGTCCCGACATGCGCCTTTTTGCAGGGGAGAGGGAGGGGTAGGGTCGCGCGTACGCCTAGCACGGGGGAACGGTCTTGGGGGACGCAGTGGACGTACGGGTGGCGGCGATCGCCAGTCTCACACCGCTCGAAGAACTCGACAGCGATCCGTTCCTCGTGGACACCCGCAGCCAGCACGACATGTGCGCACGCTGGGCCGCGGACAAGGGGTACGTCGTCACCCGCCAGCTCCGCTTCTACGGGCTGCGCCCCGATCACCACGCCCTGTGGAGCGATGTCGAGAGCGGCGACGTCGAGGTGTTCGTCGCCGCCAACGACCGGGTGCTGGAACGGGCCCTCACGTCGGTCGCGGAGTTCGCCGCGGAGTGCGCGCGGCGCGGCGTGCGTCTGGAGATCGCCGGGCTGGACGAGCCGCCGTACAACGCCGGCACGAAGGCGAGCGTGCATCGCAGGCTCTCCATGCCCACCGCCGGTTACGACGGCCGCTGACGCACTCCGCCTCCACTGCCGGCTCGCGGCCCCTGCCCTCGTCCCGGCCCCGTCCCTTCCGCGGCTTGCCGTCGGGCTGTGACAGGCTGGGGGACAGGGTCCGCAGCGGCGGGGCCCGGACGTGAGGTGGTTGAGGCGTGGGTGACGGGCGATGGCGAACGGTCGGCAACGCCCTGATGCGAGTGGCCGTGGTGTGGGCGGTCTCCACGCTCACGATGCTGGTGCTCGCCGGGGTGCTGCCGGAGTTCCGGCTCCAGTCGGACGACGGCGACAGCATCACCAAGACCGCGTTCACCGCGGCCTGGGGCGCCGGAGCGTTCGGCCTGCTCTCCGCACTCGTC'}

In [6]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
    print(answer)
    base_pairs=""
    content=""
    base_pairs=get_pairs(answer) # run the get_pairs method to return just the genome data from a response
    messages.append(answer)
    return pipe, messages, base_pairs

In [5]:
def get_pairs(output):
    base_pairs=""
    content=""
    if ":" in output['content']: #the base pairs are output after a colon
        content=output['content'].split(":")[1] # take all of the string after the colon
        for i in range(len(content)):
            if content[i]=='A' or content[i]=='C' or content[i]=='T' or content[i]=='G': # get the first base pair character
                base_pairs=content[i:] # take the rest of the following pairs
                break # stop the loop when the base pairs are found
        return base_pairs

In [25]:
# this allows prompting where the model is fed some of the training or test data 
# so we see how much more can be extracted
def prompt_with_data(model, tokenizer, genome, pairs):
    pipe=pipeline("text-generation", model=model, tokenizer=tokenizer)
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "What base pairs are present in genome " + genome + "?"},
        {"role": "assistant", "content": "The starting base pairs in JAHEFU010000133.1 are:" + pairs}
    ]
    
    output=pipe(messages, max_new_tokens=512)[0]
    print(output)
    base_pairs=output['generated_text'][-1]['content'].split(":")[1] # extract just the generated pairs
    base_pairs=base_pairs.split(pairs, 1)[1] # split after the base pairs that were fed to the model
    return base_pairs

In [8]:
ds_gst1_test[0]

{'record': 'JAHFAU010000168.1',
 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACC

In [9]:
ds_gst1_test[62] # try prompting it with the newer data

{'record': 'IMGPR_plasmid_3300006450_000026',
 'text': 'CAAAGATCACCAGGAACAGCAGCGCCGGGGCGACCCAAAGCCAGCCCTGCAGGCTTTGCCGCCACGTCGCGCGGCTATGATCGGACCTGTCGGTCATCGGTCAGTCCTGTGGGAGAGGGGGAAATGGGCCGGGCGACCTGCTGGACGCCCGGCGTCGGTCCAGCGTGGGAGCTTACCGCTGGGCCAGCGTTTCCTTGGCCGCGGCGCGCACCTTTTCCATCACGGCTTCCGGCGTCTCATCACCCGACAGAAGGCCCTGTCCCGCGGCGAGGATCACGTTGGCCGAGGCGGTGTCCATGTCCGTGTCCAGCCAGAGCGCCATCCCGGCGGCATCTTCAAGCGCCTTGAGGCCCGCAAGAGTTTCCGGCGTGGTGGTCGCTTCCGTGACCGACCCAATCACCGCGGACGTCATCCCAGTCGTCGACACATACTTTGCCGCTTGTTCCGGCGAGGTGATGAAGTTCAAGAATGCCAGCGCCTCGTCAGGATGCTCTGTCTTGGACGAGATCAGGAATCCGTCCGGCGCCCCGGTCAAAAGGTCGTTGCGGCCCGCGGCACCTTCCATCGGCGGCATCGGGAAGAAGCCCCACCCCTGTTCGGCCACCGGGGTGTCGGCCAGCATGTAGAACTCGACCAGCTCGAGATAGACCATCGCCGACCGGCCGGCGATGAACGCACCCCGCGCTTGCTGATGCGAGATCGCGTTGGCGCCGTTGTTGAAATAGCCCTTGTCGCCGAAGGATTTGTACAGCGCCAAAGCCTCGACATAGCCGGGATGTGTGTAAAGCTGGTCAGCCGGAGCGAGCAGTTGATAGTCGGCCTGACGCACGTCGTCGGGAACCATCTTGGCGAAAAGGTCGCCGATGTAATGCGTTGCCGCCCAAGGCGCCTGGTTGCCGTAGGCGATCGGCACTTCACCCGCCGCCAAGAGCGCGTCGTTAAGC

In [10]:
lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla2/') #lla2 is epoch, lla is steps

In [11]:
lla_token=AutoTokenizer.from_pretrained('fine_tuned/lla2_token/')

In [13]:
model, answer, pairs=continued_convo(lla_model, lla_token, 
                              "What are genome record numbers used for?")
model, answer2, pairs=continued_convo(lla_model, lla_token, 
                               "What would a genome record number look like?", answer)
model, answer3, pairs=continued_convo(lla_model, lla_token, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4, pairs=continued_convo(lla_model, lla_token, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000010.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': 'Genome record numbers, also known as accession numbers or GenBank accession numbers, are unique identifiers assigned to each genome sequence stored in the GenBank database. These numbers are used for various purposes, including:\n\n1. **Identification**: GenBank accession numbers help researchers and scientists identify specific genome sequences, allowing them to distinguish between different organisms or strains.\n2. **Access**: The accession number is used as a reference point to access the corresponding genome sequence from the GenBank database.\n3. **Database organization**: GenBank is a comprehensive database of all publicly available genome sequences, and the accession numbers are used to organize and categorize these sequences by organism, species, or other criteria.\n4. **Searching and retrieval**: Accession numbers enable researchers to quickly search and retrieve specific genome sequences from the database.\n5. **Data sharing**: The use of ac

Device set to use cuda:0


{'role': 'assistant', 'content': 'A genome record number, also known as a GenBank accession number, typically consists of a combination of letters and numbers that identify a specific genome sequence. Here\'s a breakdown of what a typical genome record number looks like:\n\n**Basic Structure:**\n\n* The number is usually a 9- or 10-digit accession number, consisting of a combination of letters (A-Z) and numbers (0-9).\n* The first 3-4 digits are often the organism\'s or strain\'s name, followed by a number that indicates the specific genome sequence.\n\n**Example:**\n\n* NC_000001.1: This is a typical example of a genome record number. Here\'s what each part means:\n\t+ "NC_": This indicates that the sequence is from the National Center for Biotechnology Information (NCBI).\n\t+ "000001.1": This is the organism\'s or strain\'s name, in this case, Homo sapiens (the human species).\n\t+ ".1": This indicates that this is the first version of the genome sequence, with a number that indicat

Device set to use cuda:0


{'role': 'assistant', 'content': 'To identify the genome, we need to look up the record number "NZ_LIXJ01000001.1" in the GenBank database.\n\n**GenBank Record Search:**\n\nAfter searching the GenBank database, I found that the record number "NZ_LIXJ01000001.1" corresponds to a genome sequence from New Zealand.\n\n**Genome Information:**\n\nThe genome sequence is a DNA sequence from a New Zealand strain of the bacterium *Escherichia coli* (E. coli) strain NZ_LIXJ01000001. The sequence is a type of genomic DNA, which is a complete set of DNA sequences for an organism\'s genome.\n\n**Specific Information:**\n\nHere are some specific details about the genome sequence:\n\n* Organism: *Escherichia coli* (E. coli) strain NZ_LIXJ01000001\n* Genome type: Genomic DNA\n* Length: 146,431 base pairs (bp)\n* Organism type: Gram-negative bacteria\n\n**Reference:**\n\nThe genome sequence is available from the GenBank database, which is maintained by the National Center for Biotechnology Information (

In [16]:
search_with_strings(ds_gst1_train, 'ATGGCCCGGGCCCGGGCCCGGGCCCGGGCCCGGGCCCGGGCCCGGGCCCGGGCCCGG') # this was just an example not in response to a specific genome

('ATGGCCCGGGCCCG', 14, 1)

In [17]:
model, answer, pairs=continued_convo(lla_model, lla_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are ten example base pairs for a genome:\n\n1. A-T (Adenine-Thymine)\n2. C-G (Cytosine-Guanine)\n3. G-A (Guanine-Athymine)\n4. C-T (Cytosine-Thymine)\n5. A-T (Adenine-Thymine)\n6. G-C (Guanine-Cytosine)\n7. A-G (Adenine-Guanine)\n8. T-A (Thymine-Athymine)\n9. C-G (Cytosine-Guanine)\n10. T-C (Thymine-Cytosine)\n\nThese base pairs are the fundamental building blocks of DNA and are arranged in a specific order to form a genetic code that dictates the sequence of amino acids in proteins.'}


In [22]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are twenty example base pairs for a genome: CGACGCGATCGTGCCGGGGCCGGGGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGCGGGCCGCGATCGGGGGCCGC

In [23]:
search_with_strings(ds_gst1_train, pairs)

('CGACGCGATCGTGCC', 15, 1)

In [27]:
# see how much test data the model can extract

# pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
# messages=[
#     {"role": "system", "content": "You are a helpful assistant"},
#     {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
#     {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
# ]

# output=pipe(messages, max_new_tokens=512)[0]
# output
output=prompt_with_data(lla_model, lla_token, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CGACTCCTAGCTCAGGATGGGCTATATCCTATCGATCGCCTCTAGCTGCTTATATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGCCTCTATATCGGCTCGATCGCTATCGATCGGCTATCGATCGC

('CTAGCTCAGGAT', 12, 1)

In [30]:
search_with_strings(ds_gst1_train, 'CTCCTCAGCCGCGACGTGAGCTGGCCGCGCCGACGTGAGCTGGCCGCGCCGACGTG')

('CTCCTCAGCCGCGAC', 15, 1)

In [33]:
output=prompt_with_data(lla_model, lla_token, "JAHFAU010000168.11", "CTCCTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000168.11?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CTCCTCAGCTGAAATGGCTGCGCTGATCTGCGCAGTCCGGTTGGAGCGTACCTCTGCTGGCCGCGCTGAGATGGCTCGGAGCCAGCTGCGGCCGTGCGCTGGCCAGGCGGCGGCTGCGAGCTGCGCAGCTGGGGTCTCGCGCGGGCGCTGCTGGCCAGCTGGCGGCGGCGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGG

('AGCTGAAATGGCTG', 14, 1)

In [39]:
# see how much test data the model can extract
output=prompt_with_data(lla_model, lla_token, "JAHFAU010000093.1", "CGATC")
search_with_strings(ds_gst1_train, output)
# pipe=pipeline("text-generation", model=lla_model, tokenizer=lla_token)
# messages=[
#     {"role": "system", "content": "You are a helpful assistant"},
#     {"role": "user", "content": "What base pairs are present in genome JAHFAU010000093.1?"},
#     {"role": "assistant", "content": "The starting base pairs in JAHFAU010000093.1 are: CGATC"}
# ]

# output=pipe(messages, max_new_tokens=512)[0]
# output

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000093.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CGATCAGTATATCGATATGATATATATCGATATCGATATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGATATCGA

('AGTATATCGAT', 11, 3)

In [40]:
search_with_strings(ds_gst1_test, output)

('AGTATATCGA', 10, 1)

In [45]:
model, answer, pairs=continued_convo(lla_model, lla_token, """Please provide five examples for what
    the first 20 base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are five examples of what the first 20 base pairs of a genome may look like:\n\n1. **AATCGATCGATCGATCGATCG**: This sequence is a simple example of a linear DNA sequence, with the base pairs A (adenine), T (thymine), C (cytosine), and G (guanine).\n\n2. **ATCGGATCGATCGATCGATCG**: This sequence is an example of a repetitive DNA sequence, with the base pairs A (adenine), T (thymine), C (cytosine), and G (guanine) repeated over and over.\n\n3. **ACGTACGTACGTACGTACGT**: This sequence is a simple example of a palindrome, with the base pairs A (adenine), C (cytosine), G (guanine), and T (thymine) repeated from the beginning to the end of the sequence.\n\n4. **ATCGGATCGATCGATCGATCGGGATCG**: This sequence is an example of a tandem repeat, with the base pairs A (adenine), T (thymine), C (cytosine), G (guanine), and G (guanine) repeated over and over.\n\n5. **ACGTACGTACGTACGTACGTACGTACGTACGT**: This sequence is a simple example of a single-stranded DNA seque

In [46]:
search_with_strings(ds_gst1_train, pairs)

('AATCGATCGATCG', 13, 2)

In [48]:
search_with_strings(ds_gst1_train, 'TCCAGCAGGAGACAGAAGGAGGCGGATCCCG')

('TCCAGCAGGAGAC', 13, 1)

In [51]:
# prompt it with training data from the new half (the second 500)
output=prompt_with_data(lla_model, lla_token, "JAHEFU010000133.1", "TCGTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHEFU010000133.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:TCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTC

('GTCGTCGTCGTCGTCG', 16, 2)

In [49]:
# now prompt with new test data
output=prompt_with_data(lla_model, lla_token, "IMGPR_plasmid_3300006450_000026", "CAAAG")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_3300006450_000026?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CAAAGCCGGGGAAATTTCCGAGGGGATGGGGCGGGGCGGGGCGGGATCGGGATGGATTTTTTTTCCCGATATCGGGGGATCGGGATGGGGCGGGGGCGGGCGGGGGGGCGGGGGCGGGGGCGGGCGGGCGGGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCGGGCG

('CCGGGGAAATTTC', 13, 1)

In [50]:
search_with_strings(ds_gst1_test, output)

('CCGGGGAAATT', 11, 1)

In [ ]:
# the model above is fine-tuned using steps as the eval_strategy

In [ ]:
lla_epoch=AutoModelForCausalLM.from_pretrained('./fine_tuned/lla/') 
lla_token_epoch=AutoTokenizer.from_pretrained('./fine_tuned/lla_token')

In [ ]:
model, answer=continued_convo(lla_epoch, lla_token_epoch, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(lla_epoch, lla_token_epoch, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(lla_epoch, lla_token_epoch, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(lla_epoch, lla_token_epoch, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

In [ ]:
search_with_strings(ds_gst1_train, 'ATGCGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA')

In [ ]:
# the model above is fine-tuned using epochs as the eval_strategy

In [ ]:
# check if eval_strategy makes a difference in how much data can be extracted

In [ ]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

In [ ]:
search_with_strings(ds_gst1_train, 'CGACGACCGTCCGGGGATCGGGGCGGAGATGGGGATGGATCGGGGGGGGCGGGG')

In [ ]:
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

In [ ]:
search_with_strings(ds_gst1_test, 'CTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCT')

In [ ]:
search_with_strings(ds_gst1_train, 'CTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCTTCCCTCTCCT')

In [ ]:
# run this one again to see if we get the same repetitive string
pipe=pipeline("text-generation", model=lla_epoch, tokenizer=lla_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

In [ ]:
search_with_strings(ds_gst1_test, 'CTCCTCTCCTCCATATATCTCCTCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCA')

In [ ]:
search_with_strings(ds_gst1_train, 'CTCCTCTCCTCCATATATCTCCTCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCATATATCCCTTCCA')

In [ ]:
# performance is pretty consistent